# ETL for the Datasets

In [1]:
#installing mysql connector to facilitate connection to mysql database
!pip install mysql-connector-python

In [2]:
#import the required libraries
import json
import pandas as pd
import numpy as np
import mysql.connector as connection
from sqlalchemy import create_engine
import pymysql
from urllib.parse import quote 

In [3]:
#load the language dataset
data_lang= pd.read_json("https://raw.githubusercontent.com/annexare/Countries/master/data/languages.json")
data_lang

,aa,ab,af,ak,am,an,ar,as,av,ay,...,vi,vo,wa,wo,xh,yi,yo,za,zh,zu
name,Afar,Abkhazian,Afrikaans,Akan,Amharic,Aragonese,Arabic,Assamese,Avar,Aymara,...,Vietnamese,Volapük,Walloon,Wolof,Xhosa,Yiddish,Yoruba,Zhuang,Chinese,Zulu
native,Afar,Аҧсуа,Afrikaans,Akana,አማርኛ,Aragonés,العربية,অসমীয়া,Авар,Aymar,...,Tiếng Việt,Volapük,Walon,Wollof,isiXhosa,ייִדיש,Yorùbá,Cuengh / Tôô / 壮语,中文,isiZulu
rtl,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN


In [4]:
#set index as column
df_lang = pd.DataFrame(data_lang.iloc[0])
df_lang['native']=data_lang.iloc[1]
df_lang['rtl']= data_lang.iloc[2]
df_lang


,name,native,rtl
aa,Afar,Afar,NaN
ab,Abkhazian,Аҧсуа,NaN
af,Afrikaans,Afrikaans,NaN
ak,Akan,Akana,NaN
am,Amharic,አማርኛ,NaN
...,...,...,...
yi,Yiddish,ייִדיש,1
yo,Yoruba,Yorùbá,NaN
za,Zhuang,Cuengh / Tôô / 壮语,NaN
zh,Chinese,中文,NaN


In [5]:
# to have a summarized view of the table
df_lang.describe()
# dropping rtl column has it has no significant impact in the table
df_lang.drop('rtl',inplace=True, axis =1)
df_lang

,name,native
aa,Afar,Afar
ab,Abkhazian,Аҧсуа
af,Afrikaans,Afrikaans
ak,Akan,Akana
am,Amharic,አማርኛ
...,...,...
yi,Yiddish,ייִדיש
yo,Yoruba,Yorùbá
za,Zhuang,Cuengh / Tôô / 壮语
zh,Chinese,中文


In [6]:
#resetting index
df_lang=df_lang.reset_index()
df_lang.head()

,index,name,native
0,aa,Afar,Afar
1,ab,Abkhazian,Аҧсуа
2,af,Afrikaans,Afrikaans
3,ak,Akan,Akana
4,am,Amharic,አማርኛ


In [7]:
#rename index 
df_lang = df_lang.rename(columns = {'index':'language_code'})
df_lang

,language_code,name,native
0,aa,Afar,Afar
1,ab,Abkhazian,Аҧсуа
2,af,Afrikaans,Afrikaans
3,ak,Akan,Akana
4,am,Amharic,አማርኛ
...,...,...,...
180,yi,Yiddish,ייִדיש
181,yo,Yoruba,Yorùbá
182,za,Zhuang,Cuengh / Tôô / 壮语
183,zh,Chinese,中文


In [11]:
#creating connection to mysql_ database 
engine = create_engine('mysql+mysqlconnector://root:%s@localhost:3306/database_name' % quote('password'))

#create table function
def create_table(tablename,data):
    dbConnection = engine.connect()
    dataFrame = data
    tableName =''
    try:
        frame = dataFrame.to_sql(tablename, dbConnection, if_exists='fail');
    except ValueError as vx:
        print(vx)

    except Exception as ex:   
        print(ex)

    else:
        print("Table %s created successfully."%tableName);   

    finally:
        dbConnection.close()

In [13]:
#creating language table
create_table("language",df_lang)

Table  created successfully.


In [14]:
# creating function to read table from mysql database
def read_table(query):
    dbConnection = engine.connect()
    frame = pd.read_sql(query, dbConnection);
    pd.set_option('display.expand_frame_repr', False)
    print(frame)
    dbConnection.close()

In [15]:
#reading from the language table
read_table("select language_code, name, native from test.language")

    language_code       name             native
0              aa       Afar               Afar
1              ab  Abkhazian              Аҧсуа
2              af  Afrikaans          Afrikaans
3              ak       Akan              Akana
4              am    Amharic               አማርኛ
..            ...        ...                ...
180            yi    Yiddish             ייִדיש
181            yo     Yoruba             Yorùbá
182            za     Zhuang  Cuengh / Tôô / 壮语
183            zh    Chinese                 中文
184            zu       Zulu            isiZulu

[185 rows x 3 columns]


In [16]:
#load the continents dataset
data_continent= pd.read_json("https://raw.githubusercontent.com/annexare/Countries/master/data/continents.json", typ='dictionary')
data_continent

AF           Africa
AN       Antarctica
AS             Asia
EU           Europe
NA    North America
OC          Oceania
SA    South America
dtype: object

In [17]:
#resetting the index
data_continent_idx= data_continent.reset_index()
data_continent_idx

,index,0
0,AF,Africa
1,AN,Antarctica
2,AS,Asia
3,EU,Europe
4,NA,North America
5,OC,Oceania
6,SA,South America


In [18]:
# adding column name to the respective columns
data_continent_idx.columns =['continent_code', 'continent_name']
data_continent_idx
type(data_continent_idx)

pandas.core.frame.DataFrame

In [19]:
#creating continent table
create_table("continent",data_continent_idx)

Table  created successfully.


In [20]:
#reading from the continent table
read_table("select continent_code,continent_name from test.continent")

  continent_code continent_name
0             AF         Africa
1             AN     Antarctica
2             AS           Asia
3             EU         Europe
4             NA  North America
5             OC        Oceania
6             SA  South America


In [21]:
#load the countries dataset
data_countries= pd.read_json("https://raw.githubusercontent.com/annexare/Countries/master/data/countries.json")
data_countries

,AD,AE,AF,AG,AI,AL,AM,AO,AQ,AR,...,VN,VU,WF,WS,XK,YE,YT,ZA,ZM,ZW
name,Andorra,United Arab Emirates,Afghanistan,Antigua and Barbuda,Anguilla,Albania,Armenia,Angola,Antarctica,Argentina,...,Vietnam,Vanuatu,Wallis and Futuna,Samoa,Kosovo,Yemen,Mayotte,South Africa,Zambia,Zimbabwe
native,Andorra,دولة الإمارات العربية المتحدة,افغانستان,Antigua and Barbuda,Anguilla,Shqipëria,Հայաստան,Angola,Antarctica,Argentina,...,Việt Nam,Vanuatu,Wallis et Futuna,Samoa,Republika e Kosovës,اليَمَن,Mayotte,South Africa,Zambia,Zimbabwe
phone,[376],[971],[93],[1268],[1264],[355],[374],[244],[672],[54],...,[84],[678],[681],[685],"[377, 381, 383, 386]",[967],[262],[27],[260],[263]
continent,EU,AS,AS,NA,NA,EU,AS,AF,AN,SA,...,AS,OC,OC,OC,EU,AS,AF,AF,AF,AF
capital,Andorra la Vella,Abu Dhabi,Kabul,Saint John's,The Valley,Tirana,Yerevan,Luanda,,Buenos Aires,...,Hanoi,Port Vila,Mata-Utu,Apia,Pristina,Sana'a,Mamoudzou,Pretoria,Lusaka,Harare
currency,[EUR],[AED],[AFN],[XCD],[XCD],[ALL],[AMD],[AOA],[],[ARS],...,[VND],[VUV],[XPF],[WST],[EUR],[YER],[EUR],[ZAR],[ZMW],"[USD, ZAR, BWP, GBP, AUD, CNY, INR, JPY]"
languages,[ca],[ar],"[ps, uz, tk]",[en],[en],[sq],"[hy, ru]",[pt],[],"[es, gn]",...,[vi],"[bi, en, fr]",[fr],"[sm, en]","[sq, sr]",[ar],[fr],"[af, en, nr, st, ss, tn, ts, ve, xh, zu]",[en],"[en, sn, nd]"
continents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#set index as columns
df_country = pd.DataFrame(data_countries.iloc[0])
df_country['native']=data_countries.iloc[1]
df_country['phone']= data_countries.iloc[2]
df_country['continent']=data_countries.iloc[3]
df_country['capital']= data_countries.iloc[4]
df_country['currency']=data_countries.iloc[5]
df_country['languages']= data_countries.iloc[6]
df_country['continents']= data_countries.iloc[7]
df_country

,name,native,phone,continent,capital,currency,languages,continents
AD,Andorra,Andorra,[376],EU,Andorra la Vella,[EUR],[ca],NaN
AE,United Arab Emirates,دولة الإمارات العربية المتحدة,[971],AS,Abu Dhabi,[AED],[ar],NaN
AF,Afghanistan,افغانستان,[93],AS,Kabul,[AFN],"[ps, uz, tk]",NaN
AG,Antigua and Barbuda,Antigua and Barbuda,[1268],NA,Saint John's,[XCD],[en],NaN
AI,Anguilla,Anguilla,[1264],NA,The Valley,[XCD],[en],NaN
...,...,...,...,...,...,...,...,...
YE,Yemen,اليَمَن,[967],AS,Sana'a,[YER],[ar],NaN
YT,Mayotte,Mayotte,[262],AF,Mamoudzou,[EUR],[fr],NaN
ZA,South Africa,South Africa,[27],AF,Pretoria,[ZAR],"[af, en, nr, st, ss, tn, ts, ve, xh, zu]",NaN
ZM,Zambia,Zambia,[260],AF,Lusaka,[ZMW],[en],NaN


In [23]:
#resetting index as column
df_country_idx= df_country.reset_index()
df_country_idx

,index,name,native,phone,continent,capital,currency,languages,continents
0,AD,Andorra,Andorra,[376],EU,Andorra la Vella,[EUR],[ca],NaN
1,AE,United Arab Emirates,دولة الإمارات العربية المتحدة,[971],AS,Abu Dhabi,[AED],[ar],NaN
2,AF,Afghanistan,افغانستان,[93],AS,Kabul,[AFN],"[ps, uz, tk]",NaN
3,AG,Antigua and Barbuda,Antigua and Barbuda,[1268],NA,Saint John's,[XCD],[en],NaN
4,AI,Anguilla,Anguilla,[1264],NA,The Valley,[XCD],[en],NaN
...,...,...,...,...,...,...,...,...,...
245,YE,Yemen,اليَمَن,[967],AS,Sana'a,[YER],[ar],NaN
246,YT,Mayotte,Mayotte,[262],AF,Mamoudzou,[EUR],[fr],NaN
247,ZA,South Africa,South Africa,[27],AF,Pretoria,[ZAR],"[af, en, nr, st, ss, tn, ts, ve, xh, zu]",NaN
248,ZM,Zambia,Zambia,[260],AF,Lusaka,[ZMW],[en],NaN


In [24]:
#rename column index to country_code
df_country_idx = df_country_idx.rename(columns = {'index':'country_code'})
#df_country_idx.columns =['country_code']
df_country_idx

,country_code,name,native,phone,continent,capital,currency,languages,continents
0,AD,Andorra,Andorra,[376],EU,Andorra la Vella,[EUR],[ca],NaN
1,AE,United Arab Emirates,دولة الإمارات العربية المتحدة,[971],AS,Abu Dhabi,[AED],[ar],NaN
2,AF,Afghanistan,افغانستان,[93],AS,Kabul,[AFN],"[ps, uz, tk]",NaN
3,AG,Antigua and Barbuda,Antigua and Barbuda,[1268],NA,Saint John's,[XCD],[en],NaN
4,AI,Anguilla,Anguilla,[1264],NA,The Valley,[XCD],[en],NaN
...,...,...,...,...,...,...,...,...,...
245,YE,Yemen,اليَمَن,[967],AS,Sana'a,[YER],[ar],NaN
246,YT,Mayotte,Mayotte,[262],AF,Mamoudzou,[EUR],[fr],NaN
247,ZA,South Africa,South Africa,[27],AF,Pretoria,[ZAR],"[af, en, nr, st, ss, tn, ts, ve, xh, zu]",NaN
248,ZM,Zambia,Zambia,[260],AF,Lusaka,[ZMW],[en],NaN


In [25]:
#getting a summarized view of the entire dataframe
df_country_idx.describe()

,country_code,name,native,phone,continent,capital,currency,languages,continents
count,250,250,250,250,250,250,250,250,6
unique,250,250,250,232,7,245,160,126,3
top,AD,Andorra,Andorra,[44],AF,,[EUR],[en],"[AS, EU]"
freq,1,1,1,4,58,5,35,50,4


In [26]:
#dropping cotinents column 
df_country_idx.drop('continents',inplace=True, axis =1)
df_country_idx

,country_code,name,native,phone,continent,capital,currency,languages
0,AD,Andorra,Andorra,[376],EU,Andorra la Vella,[EUR],[ca]
1,AE,United Arab Emirates,دولة الإمارات العربية المتحدة,[971],AS,Abu Dhabi,[AED],[ar]
2,AF,Afghanistan,افغانستان,[93],AS,Kabul,[AFN],"[ps, uz, tk]"
3,AG,Antigua and Barbuda,Antigua and Barbuda,[1268],NA,Saint John's,[XCD],[en]
4,AI,Anguilla,Anguilla,[1264],NA,The Valley,[XCD],[en]
...,...,...,...,...,...,...,...,...
245,YE,Yemen,اليَمَن,[967],AS,Sana'a,[YER],[ar]
246,YT,Mayotte,Mayotte,[262],AF,Mamoudzou,[EUR],[fr]
247,ZA,South Africa,South Africa,[27],AF,Pretoria,[ZAR],"[af, en, nr, st, ss, tn, ts, ve, xh, zu]"
248,ZM,Zambia,Zambia,[260],AF,Lusaka,[ZMW],[en]


In [27]:
#loading country2to 3 dataset
country_2 = pd.read_json("https://raw.githubusercontent.com/annexare/Countries/master/data/countries.2to3.json", typ="dictionary")
country_2                      

AD    AND
AE    ARE
AF    AFG
AG    ATG
AI    AIA
     ... 
YE    YEM
YT    MYT
ZA    ZAF
ZM    ZMB
ZW    ZWE
Length: 250, dtype: object

In [28]:
#resetting index
country_2 = country_2.reset_index()
country_2

,index,0
0,AD,AND
1,AE,ARE
2,AF,AFG
3,AG,ATG
4,AI,AIA
...,...,...
245,YE,YEM
246,YT,MYT
247,ZA,ZAF
248,ZM,ZMB


In [31]:
# adding column name to the respective columns
country_2.columns =['code_2_letters', 'code_3_letters']
country_2_final = pd.DataFrame(country_2, columns=['code_2_letters', 'code_3_letters'])
country_2_final

,code_2_letters,code_3_letters
0,AD,AND
1,AE,ARE
2,AF,AFG
3,AG,ATG
4,AI,AIA
...,...,...
245,YE,YEM
246,YT,MYT
247,ZA,ZAF
248,ZM,ZMB


In [32]:
#Joining or Merging the tables df_country_idx and country_2_final
df_country_finals = pd.merge(df_country_idx, country_2_final, how='left', left_on='country_code', right_on = 'code_2_letters')
df_country_finals.drop('code_2_letters',inplace=True, axis =1)
df_country_finals

,country_code,name,native,phone,continent,capital,currency,languages,code_3_letters
0,AD,Andorra,Andorra,[376],EU,Andorra la Vella,[EUR],[ca],AND
1,AE,United Arab Emirates,دولة الإمارات العربية المتحدة,[971],AS,Abu Dhabi,[AED],[ar],ARE
2,AF,Afghanistan,افغانستان,[93],AS,Kabul,[AFN],"[ps, uz, tk]",AFG
3,AG,Antigua and Barbuda,Antigua and Barbuda,[1268],NA,Saint John's,[XCD],[en],ATG
4,AI,Anguilla,Anguilla,[1264],NA,The Valley,[XCD],[en],AIA
...,...,...,...,...,...,...,...,...,...
245,YE,Yemen,اليَمَن,[967],AS,Sana'a,[YER],[ar],YEM
246,YT,Mayotte,Mayotte,[262],AF,Mamoudzou,[EUR],[fr],MYT
247,ZA,South Africa,South Africa,[27],AF,Pretoria,[ZAR],"[af, en, nr, st, ss, tn, ts, ve, xh, zu]",ZAF
248,ZM,Zambia,Zambia,[260],AF,Lusaka,[ZMW],[en],ZMB


In [33]:
#listing df_country_finals dat contained in  a list to rows
df_country_finals = df_country_finals.explode('phone')
df_country_finals = df_country_finals.explode('currency')
df_country_finals = df_country_finals.explode('languages')
df_country_finals

,country_code,name,native,phone,continent,capital,currency,languages,code_3_letters
0,AD,Andorra,Andorra,376,EU,Andorra la Vella,EUR,ca,AND
1,AE,United Arab Emirates,دولة الإمارات العربية المتحدة,971,AS,Abu Dhabi,AED,ar,ARE
2,AF,Afghanistan,افغانستان,93,AS,Kabul,AFN,ps,AFG
2,AF,Afghanistan,افغانستان,93,AS,Kabul,AFN,uz,AFG
2,AF,Afghanistan,افغانستان,93,AS,Kabul,AFN,tk,AFG
...,...,...,...,...,...,...,...,...,...
249,ZW,Zimbabwe,Zimbabwe,263,AF,Harare,INR,sn,ZWE
249,ZW,Zimbabwe,Zimbabwe,263,AF,Harare,INR,nd,ZWE
249,ZW,Zimbabwe,Zimbabwe,263,AF,Harare,JPY,en,ZWE
249,ZW,Zimbabwe,Zimbabwe,263,AF,Harare,JPY,sn,ZWE


In [34]:
#creating continent table
create_table("countries",df_country_finals)

Table  created successfully.


# Solutions to the Challenge

In [35]:
"""Question 1 List all the continents and the total number of countries in each—for
example, Africa 100, Europe 10, etc. The continent's name and country
count should be in a different column."""

#reading from the continent table
Q1=read_table("""SELECT continent_name as Continent, count(country_code) as "Country Count"
                FROM continent as a
                join countries as b
                on a.continent_code  = b.continent
                group by continent_name
                order by count(country_code)""")
Q1

       Continent  Country Count
0     Antarctica              7
1  South America             23
2        Oceania             41
3  North America             62
4           Asia             74
5         Europe             94
6         Africa            126


In [36]:
#Question 2 List all the languages and commas separated countries that speak the language.
#reading from the continent table
Q2=read_table("""select a.name as Language, group_concat(b.name separator ',')as Countries from language as a
inner join countries as b
on a.language_code = b.languages
GROUP BY a.name
order by a.name""")

       Language                                          Countries
0     Afrikaans                       Namibia,South Africa,Namibia
1      Albanian     Montenegro,Kosovo,Kosovo,Kosovo,Albania,Kosovo
2       Amharic                                           Ethiopia
3        Arabic  United Arab Emirates,Somalia,Bahrain,Iraq,Saud...
4      Armenian                                     Armenia,Cyprus
..          ...                                                ...
109       Uzbek                             Uzbekistan,Afghanistan
110       Venda                                       South Africa
111  Vietnamese                                            Vietnam
112       Xhosa                                       South Africa
113        Zulu                                       South Africa

[114 rows x 2 columns]


In [37]:
#Question 3 List all the countries and the total number of languages spoken.
Q3=read_table("""select name as Country, count(languages) as "Lng Count"
from countries
group by name
order by count(languages)
""")

                  Country  Lng Count
0              Antarctica          0
1                 Andorra          1
2    United Arab Emirates          1
3     Antigua and Barbuda          1
4                Anguilla          1
..                    ...        ...
245               Bolivia          6
246                Kosovo          8
247           Switzerland          9
248          South Africa         10
249              Zimbabwe         24

[250 rows x 2 columns]
